# Berechnungen mit Höhendifferenzen

Neben der Berechnung unter Einbeziehung von Höhendifferenzen wird auch gezeigt, wie man ein Netzwerk plotten kann.

<img src="pics/rng_height.png">

Die pandapipes-Umgebung basiert auf der Python-Bibliothek "pandas". Ein Netzwerk enthält Tabellen für jeden Komponententyp, der eingesetzt wird. Die Zeilen der Tabellen entsprechen den tatsächlich im Netzwerk vorhandenen Komponenten. Jede Zeile in einer Komponententabelle repräsentiert also eine Komponente des jeweiligen Typs. Die Spalten entsprechen den Eigenschaften des Komponententyps.

Durch das Ausführen der folgenden Code-Zellen des Notebooks werden die für das betrachtete Netz benötigten Komponententabellen erstellt. Wenn Sie weitere Informationen zu den einzelnen Parametern der Komponenten benötigen, so finden Sie diese in der pandapipes-Dokumentation unter Punkt "Datastructure and Components".


### Leeres Netzwerk

Zunächst wird pandapipes importiert. Danach wird ein leeres Netzwerk mit der Angabe des zu verwendenden Fluids hinzugefügt. Wir greifen auf die in pandapipes vorhandene Bibliothek zu und wählen Wasser als Fluid:


In [ ]:
import pandapipes as pp

In [ ]:
net = pp.create_empty_network() # create an empty network

Die <span style="color:blue">create</span>-Funktionen werden nun eingesetzt, um neue Komponenten zu erzeugen. Diese werden automatisch den entsprechenden Tabellen hinzugefügt. 

### Junctions

In [ ]:
junction1 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, height_m = 352, name="Junction 1")
junction2 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, height_m = 358, name="Junction 2")
junction3 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, height_m = 361, name="Junction 3")
junction4 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, height_m = 346, name="Junction 4")
junction5 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, height_m = 400, name="Junction 5")

Die junctions befinden sich jetzt im Netz-Container und können abgerufen werden. Einigen Parametern, die beim Funktionsaufruf nicht explizit erwähnt worden sind, wurden Standardwerte zugewiesen. Beachten Sie: Im Vergleich zum Einführungstutorial wurde den Junctions eine Höheninformation hinzugefügt. Die durch Höhenunterschiede vorhandene potentielle Energie wird bei der Hydraulikberechnung berücksichtigt.

In [ ]:
net.junction # show junction table

Im nächsten Schritt werden dem Netz weitere Komponenten hinzugefügt, angefangen beim External Grid:

### External Grid


In unserem Beispiel wird das External Grid mit der Junction-Variable junction5 verbunden. Der Druck beträgt 0.5 bar. In der späteren Auswertung wird der Druck, den Sie an junction5 abrufen, folglich ebenfalls 0.5 bar betragen.


In [ ]:
medium_pressure_grid = pp.create_ext_grid(net, junction=junction5, p_bar=0.5, t_k=293.15, fluid="water", name="Grid Connection")

net.ext_grid # show external grid table

### Rohre

Das Netzwerk enthält 4 Rohre, die zwischen je zwei Knoten verlaufen. Die Verbindungspunkte und die jeweiligen Rohrlängen sind im Diagramm zu sehen. Die Parameter sind weitgehend bereits bekannt. Beachten Sie aber, dass Rohr 4 eine andere Rauigkeit `k_mm` zugewiesen bekommt. Der Standardwert beträgt 1 mm.



In [ ]:
pipe1 = pp.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=0.545, diameter_m=0.2,  name="Pipe 1")
pipe2 = pp.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction3, length_km=0.095, diameter_m=0.15, name="Pipe 2")
pipe3 = pp.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction4, length_km=0.285, diameter_m=0.15, name="Pipe 3")
pipe4 = pp.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction5, length_km=0.43, diameter_m=0.15, k_mm=0.5, name="Pipe 4")


Damit sieht die Tabelle der Rohrkomponenten wie folgt aus:

In [ ]:
net.pipe # show pipe table

Die Parameter from_junction und to_junction geben die Orientierung des Rohrs an. Wenn die Flussrichtung tatsächlich aus Richtung from_junction in Richtung to_junction verläuft, ist das Vorzeichen der berechneten Strömungsgeschwindigkeit positiv. Andernfalls nimmt die Strömungsgeschwindigkeit negative Werte an.

### Senken


Mit Hilfe einer Senke kann eine konstante Abnahme definiert werden. An dieser Stelle werden zwei
sink-Komponente mit einem Massestrom in Höhe von 277 g/s bzw. 139 g/s definiert. Senken und Quellen
werden immer einer bestimmten Junction zugeordnet. Das Vorzeichen des Massestroms ist positiv,
wenn ein Massestrom aus dem System entnommen wird.


In [ ]:
sink = pp.create_sink(net, junction=junction4, mdot_kg_per_s=0.277, name="Sink 1")
sink = pp.create_sink(net, junction=junction3, mdot_kg_per_s=0.139, name="Sink 2")
net.sink

Wir starten jetzt die Berechnung. Wahrscheinlich ist Ihnen bekannt, dass es mehrere Modelle gibt, um den Reibungseinfluss der Rohrleitungen zu berücksichtigen. Standardmäßig wird das Modell nach Nikuradse verwendet. Eine klare Trennung zwischen laminarer und turbulenter Rohrströmung ist in pandapipes nicht implementiert. Übliche Programme bieten sonst die Möglichkeit, zwischen voll ausgebildeter, laminarer Rohrströmung und turbulenter Strömung zu unterscheiden. Laminare Strömungen sind in den üblichen pandapipes-Anwendungen nicht zu erwarten.

Dennoch ist geplant, eine entsprechende Unterscheidung in Zukunft zu implementieren. Im Pipeflow-Befehl wird an dieser Stelle explizit der Parameter für das Nikuradse-Modell gesetzt, obwohl dies die Standardeinstellung ist. 

In [ ]:
pp.pipeflow(net, friction_model ="nikuradse")

Nun lassen wir uns die Ergebnisse anzeigen:

In [ ]:
net.res_junction # calculated pressure and temperature at junctions

In [ ]:
net.res_pipe  # velocities, mass flows through pipes and other results



## Plotting

Mit Hilfe des Python-Moduls matplotlib lassen sich die Netze auch mit wenig Aufwand direkt plotten.

In [ ]:
import matplotlib
from pandapipes.plotting.simple_plot import simple_plot as sp

sp(net, plot_sinks = True)

Koordinaten werden den junctions automatisch zugewiesen, solange diese nicht explizit zugewiesen werden.